In [1]:
import sys 
import os
import shutil


In [2]:
import torch
from best_library import LoadData, Preprocessing, FeatureBuilder, DatasetSplitter, ModelBuilder, Trainer, Predictor, HyperparameterTuner, Evaluator

In [3]:
# --- 1️⃣ Configuration ---
DATASET_DIR = "../dataset"   # raw dataset
WORK_DIR = "../data"         # working directory for split dataset
BATCH_SIZE = 16
LR = 1e-4
EPOCHS = 5
IMG_SIZE = 224
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SAVE_PATH = "../models/best_model.pth"

if os.path.exists(WORK_DIR):
    shutil.rmtree(WORK_DIR)

os.makedirs(f"{WORK_DIR}/train/alpaca", exist_ok=True)
os.makedirs(f"{WORK_DIR}/train/not_alpaca", exist_ok=True)
os.makedirs(f"{WORK_DIR}/val/alpaca", exist_ok=True)
os.makedirs(f"{WORK_DIR}/val/not_alpaca", exist_ok=True)

os.makedirs("../models", exist_ok=True)


In [4]:
# ====================================================
# a) Preprocessing
# ====================================================

# Creating the transform that we are going to apply latter to the images

preprocessor = Preprocessing(img_size=IMG_SIZE)
transform = preprocessor.get_transform()

In [5]:
# ====================================================
# c) Split dataset
# ====================================================

# Filling the dataset with 2 folders train and val, each containing alpaca and not_alpaca folders

splitter = DatasetSplitter(DATASET_DIR, WORK_DIR, train_ratio=0.8)
splitter.split()

Splitting dataset...
Dataset split complete!
Training folders: ['alpaca', 'not_alpaca']
Validation folders: ['alpaca', 'not_alpaca']


In [21]:
# ====================================================
# 1️⃣ Load datasets with DataLoaders
# ====================================================

# Applying transformations to the dataset and creating two iterables for all the pictures in the training and validation

loader = LoadData(WORK_DIR, transform=transform)
train_loader, val_loader, class_names = loader.load_and_split(batch_size=BATCH_SIZE)

Classes detected: ['alpaca', 'not_alpaca']


In [22]:
# ====================================================
# b) Feature building
# ====================================================
feature_builder = FeatureBuilder()
# Example usage:
for images, labels in train_loader:
    feats = feature_builder.extract_features(images[0])

In [23]:
# ====================================================
# d) Build and train the first model
# ====================================================
num_classes = len(class_names)
model_builder = ModelBuilder(DEVICE, num_classes=num_classes)
model = model_builder.build()

trainer = Trainer(DEVICE)
trainer.train(model, train_loader, val_loader, epochs=EPOCHS, lr=LR)

Epoch 1/5 | Train Acc: 0.816 | Val Acc: 0.833
Epoch 2/5 | Train Acc: 0.989 | Val Acc: 0.894
Epoch 3/5 | Train Acc: 0.985 | Val Acc: 0.879
Epoch 4/5 | Train Acc: 0.996 | Val Acc: 0.864
Epoch 5/5 | Train Acc: 0.989 | Val Acc: 0.924


0.9242424242424242

In [24]:
# ====================================================
# e) Hyperparameter tuning
# ====================================================
param_grid = {
    "lr": [1e-3, 1e-4],
    "epochs": [3, 5]
}

tuner = HyperparameterTuner(param_grid, DEVICE)
best_params, best_acc = tuner.tune(train_loader, val_loader, save_path=SAVE_PATH)
print(f"Best hyperparameters: {best_params}, Best validation accuracy: {best_acc:.3f}")

Starting hyperparameter tuning with 4 combinations...

--- Trial 1/4: {'lr': 0.001, 'epochs': 3} ---
Epoch 1/3 | Train Acc: 0.655 | Val Acc: 0.545
Epoch 2/3 | Train Acc: 0.762 | Val Acc: 0.591
Epoch 3/3 | Train Acc: 0.897 | Val Acc: 0.833
Trial finished. Validation accuracy: 0.833

--- Trial 2/4: {'lr': 0.001, 'epochs': 5} ---
Epoch 1/5 | Train Acc: 0.667 | Val Acc: 0.561
Epoch 2/5 | Train Acc: 0.816 | Val Acc: 0.576
Epoch 3/5 | Train Acc: 0.851 | Val Acc: 0.727
Epoch 4/5 | Train Acc: 0.908 | Val Acc: 0.636
Epoch 5/5 | Train Acc: 0.973 | Val Acc: 0.742
Trial finished. Validation accuracy: 0.742

--- Trial 3/4: {'lr': 0.0001, 'epochs': 3} ---
Epoch 1/3 | Train Acc: 0.759 | Val Acc: 0.773
Epoch 2/3 | Train Acc: 0.977 | Val Acc: 0.939
Epoch 3/3 | Train Acc: 0.996 | Val Acc: 0.939
Trial finished. Validation accuracy: 0.939

--- Trial 4/4: {'lr': 0.0001, 'epochs': 5} ---
Epoch 1/5 | Train Acc: 0.808 | Val Acc: 0.864
Epoch 2/5 | Train Acc: 0.985 | Val Acc: 0.894
Epoch 3/5 | Train Acc: 0.996 

In [25]:
# ====================================================
# f) Evaluate model
# ====================================================
evaluator = Evaluator(DEVICE)
best_model = Predictor(DEVICE, class_names).load_model(SAVE_PATH)
val_accuracy = evaluator.evaluate(best_model, val_loader)
print(f"Validation accuracy of the best model: {val_accuracy:.3f}")

Validation accuracy of the best model: 0.879


In [26]:
# ====================================================
# g) Predict new images 
# ====================================================
predictor = Predictor(DEVICE, class_names)
image_path = "../dog2.jpg"
label, confidence = predictor.predict(image_path, best_model, transform)
print(f"Predicted label: {label}, Confidence: {confidence:.3f}")

Predicted label: not_alpaca, Confidence: 0.965
